# Benchmark linear advection with EnKF

In this notebook, we are interested in the sequential inference 



References: 


[1] Evensen, G., 1994. Sequential data assimilation with a nonlinear quasi‐geostrophic model using Monte Carlo methods to forecast error statistics. Journal of Geophysical Research: Oceans, 99(C5), pp.10143-10162.

[2] Asch, M., Bocquet, M. and Nodet, M., 2016. Data assimilation: methods, algorithms, and applications. Society for Industrial and Applied Mathematics.

[3] Bishop, C.H., Etherton, B.J. and Majumdar, S.J., 2001. Adaptive sampling with the ensemble transform Kalman filter. Part I: Theoretical aspects. Monthly weather review, 129(3), pp.420-436. 

[4] Lorenz, E.N., 1963. Deterministic nonperiodic flow. Journal of atmospheric sciences, 20(2), pp.130-141.

[5] Spantini, A., Baptista, R. and Marzouk, Y., 2019. Coupling techniques for nonlinear ensemble filtering. arXiv preprint arXiv:1907.00389.

### The basic steps
To carry out sequential inference in `AdaptiveTransportMap`, we need to carry out a few basic steps:
* **Specify the problem**: Define the state-space model: initial condition, dynamical and observation models (including process and observation noise)
* **Specify the inflation parameters**: Determine the levels of covariance inflation to properly balance the dynamical system and the observations from the truth system
* **Specify the filter**: Choose the ensemble filter to assimilate the observations in the state estimate
* **Perform the sequential inference**: Perform the sequential inference

We will go through all of these here.

In [1]:
using Distributed

In [2]:
addprocs(2)

3-element Vector{Int64}:
 2
 3
 4

In [3]:
@everywhere begin
    using Revise
    using LinearAlgebra
    using InvariantDA
    using TransportBasedInference
    using Statistics
    using Distributions
    using PDMats
    using FFTW
    using OrdinaryDiffEq
    using ProgressMeter
    using JLD
end

[ Info: Precompiling InvariantDA [89772154-e7dd-42c1-9ff1-66f2d667c9c9]


In [4]:
@everywhere path = "/Users/mathieu/Documents/InvariantDA.jl/notebooks/linear_advection/"

In [5]:
@everywhere begin
Nx = 128
Ny = 32
end

In [6]:
@everywhere begin
Δtdyn = 0.05
Δtobs = 0.05
end

In [7]:
@everywhere begin
t0 = 0.0
Tf = 2000
tf = Tf*Δtobs

Tmetric = 1000
tmetric = Tmetric*Δtobs
end

In [8]:
@everywhere π0 = MvNormal(zeros(Nx), Matrix(1.0*I, Nx, Nx))

In [9]:
@everywhere begin
# Parameters to set-up the linear advection problem
L = 1.0
c = 1.0
Δx = L/Nx
xgrid = collect(Δx*(0:1:Nx-1))
    
params = Dict("N" => Nx,
              "L" => L,
              "c" => c,
              "plan" => plan_rfft(zeros(Nx)),
              "c∂x" => map(k-> k == Nx÷2 ? 0.0*im : c*(2*π*im*k/L), 0:Nx÷2))
end

In [10]:
cfl = c*Δtdyn/Δx

6.4

In [11]:
# m0 = 1.0

In [12]:
@everywhere x0 = irfft((randn(Nx÷2+1) + im*randn(Nx÷2+1)).* map(k-> exp(-0.5*k), 1:Nx÷2+1), Nx)/Δx;

In [13]:
@everywhere begin 
h(x, t) = x[1:4:end]
H = Matrix(1.0*I, Nx, Nx)[1:4:end,:]
F = StateSpace((du, u, p, t) -> spectral_linear_advection!(du, u, params, t), h)
end

In [14]:
@everywhere begin
    σx = 1e-16
    σy = 0.5

    ϵx = AdditiveInflation(Nx, zeros(Nx), σx)
    ϵy = AdditiveInflation(Ny, zeros(Ny), σy)
end

In [15]:
@everywhere begin
    Δ = 4
    yidx = 1:Δ:Nx
    idx = vcat(collect(1:length(yidx))', collect(yidx)')

    # Create Localization structure
    Gxx(i,j) = periodicmetric!(i,j, Nx)
    Gxy(i,j) = periodicmetric!(i,yidx[j], Nx)
    Gyy(i,j) = periodicmetric!(yidx[i],yidx[j], Nx)
end

In [16]:
@everywhere model = Model(Nx, Ny, Δtdyn, Δtobs, ϵx, ϵy, π0, 0, 0, 0, F);

In [17]:
@everywhere begin
ϕm = ones(Nx)/sqrt(Nx);
Qm = qr(ϕm).Q;
Qperp = Qm*Matrix(1.0*I, Nx, Nx)[:,2:end];
end

In [18]:
@everywhere m0 = dot(ϕm, x0)

In [22]:
@everywhere begin
# data = generate_data_rfft(model, x0, Tf);
# save(path*"linear_advection_data_benchmark_enkf.jld", "data", data)
data = load(path*"linear_advection_data_benchmark_enkf.jld", "data")
end

Define the different filters

In [23]:
@everywhere Nrun = 1

In [24]:
@everywhere function custom_output_metrics(data::SyntheticData, model::Model, Tbegin::Int64, Tend::Int64, statehist::Array{Array{Float64,2},1})
    Ne = size(statehist[1],2)

    # Post_process compute the statistics (mean, median, and
    # standard deviation) of the RMSE, spread and coverage
    # probability over (J-T_BurnIn) assimilation steps.

    # enshist contains the initial condition, so one more element
    idx_xt = Tbegin+1:Tend

    idx_ens = Tbegin+2:Tend+1

    # Compute root mean square error statistics
    Rmse, Rmse_med, Rmse_mean, Rmse_std = metric_hist(rmse, data.xt[:,idx_xt], statehist[idx_ens])
    # Compute ensemble spread statistics
    Spread, Spread_med, Spread_mean, Spread_std = metric_hist(spread, statehist[idx_ens])
    # Compute quantile information
    qinf, qsup = TransportBasedInference.quant(statehist[idx_ens])

    # Compute coverage probability statistics

    Covprob = zeros(length(idx_xt))
    b = zeros(Bool, model.Nx)
    for (i,idx) in enumerate(idx_xt)
        for j=1:model.Nx
        b[j] = (qsup[j,i] >= data.xt[j,idx] >= qinf[j,i])
        end
        Covprob[i] = deepcopy(mean(b))
    end

    Covprob_med  = median(Covprob)
    Covprob_mean = mean(Covprob)
    Covprob_std  = std(Covprob)

    Metric = Metrics(Ne, Rmse, Rmse_med, Rmse_mean, Rmse_std, Spread, Spread_med,
                    Spread_mean, Spread_std, Covprob, Covprob_med,
                    Covprob_mean, Covprob_std)
    return Metric
end

In [25]:
@everywhere begin
    β_array = collect(0.95:0.01:1.05)
    L_array = collect(3:1:40)
    Ne_array = [40, 50, 60, 80, 100, 120, 150, 200, 300, 500]
end

In [26]:
# Distribution for the mass uncertainty
@everywhere πm0 = Uniform(0.9, 1.1)

In [ ]:
@everywhere begin
    metric_enkf_list = []
    metric_enkf_mass_list = []
end

@sync @distributed for Ne in Ne_array
    @show Ne
    metric_enkf_Ne = Metrics[]
    metric_enkf_mass_Ne = Metrics[]

    X = zeros(model.Ny + model.Nx, Ne)

    # Generate the initial conditions for the state.
    
    for i=1:Ne
        xsmoothi = irfft((randn(Nx÷2+1) + im*randn(Nx÷2+1)).* map(k-> exp(-0.5*k^(1.3)), 1:Nx÷2+1), Nx)/Δx;
        mi = m0*rand(πm0)

        X[Ny+1:Ny+Nx,i] = ϕm*mi + (I - ϕm*ϕm')*xsmoothi
    end

    for β in β_array
        for Lrad in L_array
            Loc = Localization(Lrad, Gxx, Gxy, Gyy)
        
            enkf = LocEnKF(model.ϵy, model.Δtdyn, model.Δtobs, Loc, H)
            ϵxβ = MultiAddInflation(Nx, β, zeros(Nx), σx)
            Xenkf = seqassim_rfft(F, data, Tf, ϵxβ, enkf, deepcopy(X), model.Ny, model.Nx, t0)
        
            metric_enkf = custom_output_metrics(data, model, Tmetric, Tf,  Xenkf)
            push!(metric_enkf_Ne, deepcopy(metric_enkf))
                
            Xenkf_mass = seqassim_rfft_project_mass(F, data, Tf, ϵxβ, ϕm, enkf, deepcopy(X), model.Ny, model.Nx, t0)

            metric_enkf_mass = custom_output_metrics(data, model, Tmetric, Tf,  Xenkf_mass)
            push!(metric_enkf_mass_Ne, deepcopy(metric_enkf_mass))
        end
    end
    
    push!(metric_enkf_list, deepcopy(metric_enkf_Ne))    
    push!(metric_enkf_mass_list, deepcopy(metric_enkf_mass_Ne))
            
    save(path*"benchmark_linear_advection_enkf.jld", "metric", metric_enkf_list)
    save(path*"benchmark_linear_advection_enkf_mass.jld", "metric", metric_enkf_mass_list)
        
    print("Done "*string(Ne))
end

      From worker 4:	Ne = 200
      From worker 2:	Ne = 40
      From worker 3:	Ne = 100
